# Caco-2 (Cell Effective Permeability), Wang et al.

### Dataset Description: The human colon epithelial cancer cell line, Caco-2, is used as an in vitro model to simulate the human intestinal tissue. The experimental result on the rate of drug passing through the Caco-2 cells can approximate the rate at which the drug permeates through the human intestinal tissue.

### Task Description: Regression. Given a drug SMILES string, predict the Caco-2 cell effective permeability.

### Dataset Statistics: 906 drugs.

### Metric: MAE

## Leaderboard

| Rank | Model                       | Contact        | Link          | #Params   | MAE           |
|------|-----------------------------|----------------|---------------|-----------|---------------|
| 1    | MapLight                    | Jim Notwell    | GitHub, Paper | N/A       | 0.276 ± 0.005 |
| 2    | BaseBoosting                | Andrew Li      | GitHub, Paper | 365,713   | 0.285 ± 0.005 |
| 3    | MolMapNet-D                 | Shen Wan Xiang | GitHub, Paper | 407,617   | 0.287 ± 0.005 |
| 4    | MapLight + GNN              | Jim Notwell    | GitHub, Paper | N/A       | 0.287 ± 0.005 |
| 5    | XGBoost                     | Andrew Li      | GitHub, Paper | 12        | 0.289 ± 0.011 |
| 6    | Basic ML                    | Nilavo Boral   | GitHub, Paper | N/A       | 0.321 ± 0.005 |
| 7    | Chemprop-RDKit              | Kyle Swanson   | GitHub, Paper | N/A       | 0.330 ± 0.024 |
| 8    | Euclia ML model             | Euclia         | GitHub, Paper | 50        | 0.341 ± 0.004 |
| 9    | Chemprop                    | Kyle Swanson   | GitHub, Paper | N/A       | 0.344 ± 0.015 |
| 10   | RDKit2D + MLP (DeepPurpose) | Kexin Huang    | GitHub, Paper | 633,409   | 0.393 ± 0.024 |
| 11   | AttentiveFP                 | Kexin Huang    | GitHub, Paper | 300,806   | 0.401 ± 0.032 |
| 12   | CNN (DeepPurpose)           | Kexin Huang    | GitHub, Paper | 226,625   | 0.446 ± 0.036 |
| 13   | ContextPred                 | Kexin Huang    | GitHub, Paper | 2,067,053 | 0.502 ± 0.036 |
| 14   | NeuralFP                    | Kexin Huang    | GitHub, Paper | 480,193   | 0.530 ± 0.102 |
| 15   | AttrMasking                 | Kexin Huang    | GitHub, Paper | 2,067,053 | 0.546 ± 0.052 |
| 16   | GCN                         | Kexin Huang    | GitHub, Paper | 191,810   | 0.599 ± 0.104 |
| 17   | Morgan + MLP (DeepPurpose)  | Kexin Huang    | GitHub, Paper | 1,477,185 | 0.908 ± 0.060 |

In [1]:
import pandas as pd
from deepmol.pipeline import Pipeline

2023-11-08 14:42:24.939300: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 14:42:24.973119: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-08 14:42:24.973908: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 14:42:25.539200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.


In [2]:
# read results
results = pd.read_csv('caco/tdc_test_set_results.txt', sep=',', header=None, dtype={0: int, 1: float, 2: float})
# set columns
results.columns = ['trial_id', 'mean', 'std']
results
# order res

,trial_id,mean,std
0,0,0.458,0.004
1,1,0.603,0.058
2,2,0.570,0.023
3,4,0.464,0.008
4,5,1.366,0.142
5,6,0.463,0.048
6,7,0.580,0.042
7,8,0.773,0.153
8,9,0.620,0.087
9,10,0.357,0.008


In [4]:
# order results by mean (std in case of tie)
results = results.sort_values(by=['mean', 'std'], ascending=True)
results

,trial_id,mean,std
9,10,0.357,0.008
11,12,0.357,0.011
14,22,0.361,0.021
38,64,0.370,0.018
18,33,0.372,0.032
10,11,0.372,0.041
13,21,0.376,0.016
21,36,0.382,0.019
24,41,0.387,0.020
20,35,0.392,0.010


In [6]:
# load best trial pipeline (rank #10)
best_trial_id = int(results.iloc[0]['trial_id'])
pipeline = Pipeline.load(f"caco/trial_{best_trial_id}/")

[14:43:35] Initializing Normalizer


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpvy5i7n9v/model.pkl'